In [1]:
import numpy as np
import pandas as pd


In [3]:
df=pd.read_csv('/content/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [10]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values


In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X=sc.fit_transform(X)
X.shape

(768, 8)

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [81]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout

In [14]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - accuracy: 0.4547 - loss: 0.7218 - val_accuracy: 0.5584 - val_loss: 0.6756
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6615 - loss: 0.6542 - val_accuracy: 0.7338 - val_loss: 0.6124
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6945 - loss: 0.6113 - val_accuracy: 0.7338 - val_loss: 0.5673
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7037 - loss: 0.5860 - val_accuracy: 0.7532 - val_loss: 0.5353
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7430 - loss: 0.5466 - val_accuracy: 0.7662 - val_loss: 0.5100
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7635 - loss: 0.5339 - val_accuracy: 0.7532 - val_loss: 0.4910
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7936 - loss: 0.4937 - val_accuracy: 0.7727 - val_loss: 0.4742
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7650 - loss: 0.4849 - val_accuracy: 0.7922 - val_los

In [ ]:
# now using the kerastuner library to get the best paramaters
# 1.) how to select appropriate optimiser
# 2.) number of nodes in a layer
# 3.) how to select number of layers
# 4.) all in all onr model

In [16]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 11.7 MB/s eta 0:00:00


In [21]:
import kerastuner as kt

In [24]:
def build_model(hp):
  model=Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [25]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [27]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [28]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [29]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
model.fit(  X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test),initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.7416 - loss: 0.5626 - val_accuracy: 0.8117 - val_loss: 0.4997
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7293 - loss: 0.5417 - val_accuracy: 0.8052 - val_loss: 0.4834
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7579 - loss: 0.5011 - val_accuracy: 0.8182 - val_loss: 0.4722
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7301 - loss: 0.5413 - val_accuracy: 0.8247 - val_loss: 0.4629
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7341 - loss: 0.5138 - val_accuracy: 0.8247 - val_loss: 0.4557
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7531 - loss: 0.4765 - val_accuracy: 0.8247 - val_loss: 0.4491
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7560 - loss: 0.4877 - val_accuracy: 0.8247 - val_loss: 0.4443
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7672 - loss: 0.4710 - val_accuracy: 0.83

In [ ]:
#  now for the number of the neurons

In [36]:
def build_model(hp):
  model=Sequential()

  units=hp.Int('units',min_value=8,max_value=128,step=1)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  # optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [37]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='project')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
tuner.get_best_hyperparameters()[0].values

{'units': 100}

In [40]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [41]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test),initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.7557 - loss: 0.5135 - val_accuracy: 0.8052 - val_loss: 0.4580
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7743 - loss: 0.4810 - val_accuracy: 0.8247 - val_loss: 0.4473
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7638 - loss: 0.4687 - val_accuracy: 0.8182 - val_loss: 0.4403
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7721 - loss: 0.4667 - val_accuracy: 0.8117 - val_loss: 0.4362
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7764 - loss: 0.4525 - val_accuracy: 0.8182 - val_loss: 0.4338
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7706 - loss: 0.4594 - val_accuracy: 0.8182 - val_loss: 0.4323
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7744 - loss: 0.4684 - val_accuracy: 0.8182 - val_loss: 0.4319
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7831 - loss: 0.4395 - val_accuracy: 0.81

In [ ]:
# for the numebr of t he layers int he model

In [42]:
def build_model(hp):
  model=Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(32,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))


  # optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [43]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='project')

Reloading Tuner from project/untitled_project/tuner0.json


In [44]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [45]:
tuner.get_best_hyperparameters()[0].values

{'units': 87}

In [54]:
def build_model1(hp):
  model=Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(32,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))


  # optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [55]:
tuner=kt.RandomSearch(build_model1,objective='val_accuracy',max_trials=5,directory='project2')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [57]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [58]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [59]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test),initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - accuracy: 0.7406 - loss: 0.4926 - val_accuracy: 0.8312 - val_loss: 0.4221
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7726 - loss: 0.4768 - val_accuracy: 0.8312 - val_loss: 0.4238
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7446 - loss: 0.4990 - val_accuracy: 0.8312 - val_loss: 0.4261
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7704 - loss: 0.4612 - val_accuracy: 0.8312 - val_loss: 0.4228
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7928 - loss: 0.4330 - val_accuracy: 0.8182 - val_loss: 0.4284
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7531 - loss: 0.4768 - val_accuracy: 0.8377 - val_loss: 0.4268
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7769 - loss: 0.4336 - val_accuracy: 0.8377 - val_loss: 0.4294
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7737 - loss: 0.4596 - val_accuracy: 0.81

In [ ]:
# all in one

In [83]:
def build_model(hp):
  model=Sequential()
  counter=0

  for i in range(hp.Int('layers',min_value=1,max_value=10)):
    if counter==0:
      model.add(
          Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=2),
              activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),
              input_dim=8
          )
      )
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(
          Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=2),
              activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])
          )
      )
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(
      optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta']),
      loss='binary_crossentropy',
      metrics=['accuracy']
  )

  return model

In [84]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='project4')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [85]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 25s]
val_accuracy: 0.6948052048683167

Best val_accuracy So Far: 0.6948052048683167
Total elapsed time: 00h 01m 16s


In [70]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [71]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test),initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.7707 - loss: 0.4820 - val_accuracy: 0.8117 - val_loss: 0.4280
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7520 - loss: 0.4917 - val_accuracy: 0.8182 - val_loss: 0.4240
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7350 - loss: 0.5177 - val_accuracy: 0.8182 - val_loss: 0.4224
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7485 - loss: 0.5011 - val_accuracy: 0.8117 - val_loss: 0.4238
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7753 - loss: 0.4713 - val_accuracy: 0.8117 - val_loss: 0.4213
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7654 - loss: 0.4797 - val_accuracy: 0.8117 - val_loss: 0.4236
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7717 - loss: 0.4833 - val_accuracy: 0.8182 - val_loss: 0.4226
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7645 - loss: 0.4978 - val_accuracy: 0.80

In [86]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [87]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='project4')

Reloading Tuner from project4/untitled_project/tuner0.json
